In [1]:
import numpy as np
import copy
import keras.models

from MCTS import MCTS
from Game import Game 
from model import AmoebaZeroModel
import params


Using TensorFlow backend.


In [ ]:
HISTORY_SIZE     = params.PIPELINE_HISTORY_SIZE
BATCH_SIZE       = params.PIPELINE_BATCH_SIZE
TRAINING_LOOP_NR = params.PIPELINE_TRAINING_LOOP_NR
SELFPLAY_NR      = params.PIPELINE_SELFPLAY_NR
EVAL_NR          = params.PIPELINE_EVAL_NR
THRESHOLD        = params.PIPELINE_THRESHOLD

class Pipeline:
    def __init__(self, verbose = 0):
        self.verbose = verbose
        
        self.history_input = np.zeros( (HISTORY_SIZE,15,15,4))
        self.history_p     = np.zeros( (HISTORY_SIZE,225))
        self.history_v     = np.zeros( (HISTORY_SIZE))
        self.batch_input   = np.zeros( (BATCH_SIZE,15,15,4))
        self.batch_p       = np.zeros( (BATCH_SIZE,225))
        self.batch_v       = np.zeros( (BATCH_SIZE))
        self.next_index    = 0
        self.history_nr    = 0
        self.version       = 0
        
        self.model_trained = AmoebaZeroModel()
        self.model_trained.model.save("./models/saved_model_0.h5")
        
        self.model_best       = AmoebaZeroModel()
        self.model_best.model = keras.models.load_model("./models/saved_model_0.h5")
    
    def main_loop(self):
        nr = 0
        while True:
            print("Loop - ", nr)
            self.self_play()
            self.train()
            if self.evaluate_trained():
                self.version += 1
                self.model_trained.model.save("./models/saved_model_"+str(self.version)+".h5")
                self.model_best.model = keras.models.load_model("./models/saved_model_"+str(self.version)+".h5")
                print("New model saved. Version: ", self.version)
    
    def self_play(self):
        if self.verbose >= 1:
            print("Self play")
        for i in range(SELFPLAY_NR):
            if self.verbose >=2:
                print(i, "start_index = ", self.next_index)
            start_index = self.next_index
            g = Game()
            mcts = MCTS(g, self.model_best.evaluate)
            nr = 0
            while (g.winner==0):
                nr += 1
                x,y,n = mcts.select_move()                
                g.fill_grids_for_nn(self.history_input,self.next_index)
                mcts.fill_p(self.history_p,self.next_index)
                self.next_index = (self.next_index + 1) % HISTORY_SIZE
                if self.history_nr<HISTORY_SIZE:
                    self.history_nr += 1
                g.move(x,y)
                mcts.reinit(g,n)
                if self.verbose>=3:
                    print(nr, "- move: ", x, y, "; next_index: ", self.next_index)
                if i==0 and self.verbose>=4:
                    g.print()
            v = g.get_final_value()

            if self.verbose >= 2:
                print("Result:", g.winner)
            
            i = start_index
            while i != self.next_index:
                self.history_v[i] = v
                i = (i+1)%HISTORY_SIZE
    
    def train(self):
        if self.verbose >= 1:
            print("Train")
        for i in range(TRAINING_LOOP_NR):
            if self.verbose >= 2:
                print("i: " + str(i))
            
            for j in range(BATCH_SIZE):
                ind = np.random.randint(0,self.history_nr)
                self.batch_input[j] = self.history_input[ind]
                self.batch_p[j]     = self.history_p[ind]
                self.batch_v[j]     = self.history_v[ind]
            self.model_trained.train(self.batch_input,[self.batch_p,self.batch_v],BATCH_SIZE)           
            
    def evaluate_trained(self):
        if self.verbose >= 1:
            print("Evaluate")

        balance = 0 #+ if trained, - if best won more
        
        #best begins
        for i in range(EVAL_NR//2):
            if self.verbose >= 2:
                print("i: ",i)
            g  = Game()
            nr = 0
            while (g.winner==0):
                if nr%2==0:
                    mcts = MCTS(g, self.model_best.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                else:
                    mcts = MCTS(g, self.model_trained.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                nr+=1
                if i==0 and self.verbose>=4:
                    g.print()
                
            if g.winner == 1:
                balance -= 1
            elif g.winner == 2:
                balance += 1

            if self.verbose >= 2:
                print("Result:", g.winner)

        #trained begins
        for i in range(EVAL_NR//2):
            if self.verbose >= 2:
                print("i: ",i)
            g  = Game()
            nr = 0
            while (g.winner==0):
                if nr%2==0:
                    mcts = MCTS(g, self.model_trained.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                else:
                    mcts = MCTS(g, self.model_best.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                nr+=1
                if i==0 and self.verbose>=4:
                    g.print()
            if g.winner == 1:
                balance += 1
            elif g.winner == 2:
                balance -= 1

            if self.verbose >= 2:
                print("Result:", g.winner)

                
        if self.verbose >= 1:
            print("Total result: ",balance," Thr.: ",THRESHOLD)
                
        return balance >= THRESHOLD
           
        
        

In [ ]:
pl = Pipeline(verbose = 2)
pl.main_loop()


Loop -  0
Self play
0 start_index =  0
Result: 1
1 start_index =  67
Result: 2
2 start_index =  175
Result: 2
3 start_index =  249
Result: 2
4 start_index =  297
Result: 2
5 start_index =  367
Result: 2
6 start_index =  457
Result: 1
7 start_index =  576
Result: 2
8 start_index =  656
Result: 1
9 start_index =  691
Result: 1
Train
i: 0
Epoch 1/1
16/16 [==============================] - 1s 35ms/step - loss: 8.8905 - out_p_loss: 5.5887 - out_v_loss: 0.3320 - out_p_categorical_crossentropy: 5.5887 - out_v_mean_squared_error: 0.3320
i: 1
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 8.8480 - out_p_loss: 5.6167 - out_v_loss: 0.2961 - out_p_categorical_crossentropy: 5.6167 - out_v_mean_squared_error: 0.2961
i: 2
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 8.7306 - out_p_loss: 5.5971 - out_v_loss: 0.2316 - out_p_categorical_crossentropy: 5.5971 - out_v_mean_squared_error: 0.2316
i: 3
Epoch 1/1
16/16 [==============================] - 0s 1ms/

i: 39
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 7.6436 - out_p_loss: 5.4668 - out_v_loss: 0.1639 - out_p_categorical_crossentropy: 5.4668 - out_v_mean_squared_error: 0.1639
i: 40
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 7.4960 - out_p_loss: 5.4406 - out_v_loss: 0.0613 - out_p_categorical_crossentropy: 5.4406 - out_v_mean_squared_error: 0.0613
i: 41
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 7.4440 - out_p_loss: 5.3492 - out_v_loss: 0.1191 - out_p_categorical_crossentropy: 5.3492 - out_v_mean_squared_error: 0.1191
i: 42
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 7.4247 - out_p_loss: 5.3769 - out_v_loss: 0.0902 - out_p_categorical_crossentropy: 5.3769 - out_v_mean_squared_error: 0.0902
i: 43
Epoch 1/1
16/16 [==============================] - 0s 990us/step - loss: 7.4884 - out_p_loss: 5.4242 - out_v_loss: 0.1247 - out_p_categorical_crossentropy: 5.4242 - out_v_mean_squared_er

i: 80
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.7434 - out_p_loss: 5.2910 - out_v_loss: 0.0418 - out_p_categorical_crossentropy: 5.2910 - out_v_mean_squared_error: 0.0418
i: 81
Epoch 1/1
16/16 [==============================] - 0s 982us/step - loss: 6.8600 - out_p_loss: 5.4071 - out_v_loss: 0.0534 - out_p_categorical_crossentropy: 5.4071 - out_v_mean_squared_error: 0.0534
i: 82
Epoch 1/1
16/16 [==============================] - 0s 958us/step - loss: 6.7527 - out_p_loss: 5.3275 - out_v_loss: 0.0368 - out_p_categorical_crossentropy: 5.3275 - out_v_mean_squared_error: 0.0368
i: 83
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.6921 - out_p_loss: 5.2732 - out_v_loss: 0.0415 - out_p_categorical_crossentropy: 5.2732 - out_v_mean_squared_error: 0.0415
i: 84
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.8048 - out_p_loss: 5.3776 - out_v_loss: 0.0605 - out_p_categorical_crossentropy: 5.3776 - out_v_mean_squared_

i: 16
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.4795 - out_p_loss: 5.2967 - out_v_loss: 0.1018 - out_p_categorical_crossentropy: 5.2967 - out_v_mean_squared_error: 0.1018
i: 17
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.5101 - out_p_loss: 5.3421 - out_v_loss: 0.0942 - out_p_categorical_crossentropy: 5.3421 - out_v_mean_squared_error: 0.0942
i: 18
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.5112 - out_p_loss: 5.3709 - out_v_loss: 0.0736 - out_p_categorical_crossentropy: 5.3709 - out_v_mean_squared_error: 0.0736
i: 19
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.4908 - out_p_loss: 5.3310 - out_v_loss: 0.1002 - out_p_categorical_crossentropy: 5.3310 - out_v_mean_squared_error: 0.1002
i: 20
Epoch 1/1
16/16 [==============================] - 0s 990us/step - loss: 6.6263 - out_p_loss: 5.4242 - out_v_loss: 0.1494 - out_p_categorical_crossentropy: 5.4242 - out_v_mean_squared_er

i: 57
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.3114 - out_p_loss: 5.3948 - out_v_loss: 0.0708 - out_p_categorical_crossentropy: 5.3948 - out_v_mean_squared_error: 0.0708
i: 58
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.2322 - out_p_loss: 5.3760 - out_v_loss: 0.0150 - out_p_categorical_crossentropy: 5.3760 - out_v_mean_squared_error: 0.0150
i: 59
Epoch 1/1
16/16 [==============================] - 0s 948us/step - loss: 6.1695 - out_p_loss: 5.3035 - out_v_loss: 0.0292 - out_p_categorical_crossentropy: 5.3035 - out_v_mean_squared_error: 0.0292
i: 60
Epoch 1/1
16/16 [==============================] - 0s 823us/step - loss: 6.1532 - out_p_loss: 5.3077 - out_v_loss: 0.0132 - out_p_categorical_crossentropy: 5.3077 - out_v_mean_squared_error: 0.0132
i: 61
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 6.3059 - out_p_loss: 5.4298 - out_v_loss: 0.0482 - out_p_categorical_crossentropy: 5.4298 - out_v_mean_squared_

...

16/16 [==============================] - 0s 882us/step - loss: 5.6415 - out_p_loss: 5.4180 - out_v_loss: 0.0391 - out_p_categorical_crossentropy: 5.4180 - out_v_mean_squared_error: 0.0391
i: 32
Epoch 1/1
16/16 [==============================] - 0s 858us/step - loss: 5.5992 - out_p_loss: 5.3672 - out_v_loss: 0.0477 - out_p_categorical_crossentropy: 5.3672 - out_v_mean_squared_error: 0.0477
i: 33
Epoch 1/1
16/16 [==============================] - 0s 861us/step - loss: 5.5757 - out_p_loss: 5.3770 - out_v_loss: 0.0144 - out_p_categorical_crossentropy: 5.3770 - out_v_mean_squared_error: 0.0144
i: 34
Epoch 1/1
16/16 [==============================] - 0s 821us/step - loss: 5.5788 - out_p_loss: 5.3357 - out_v_loss: 0.0589 - out_p_categorical_crossentropy: 5.3357 - out_v_mean_squared_error: 0.0589
i: 35
Epoch 1/1
16/16 [==============================] - 0s 839us/step - loss: 5.6372 - out_p_loss: 5.3712 - out_v_loss: 0.0818 - out_p_categorical_crossentropy: 5.3712 - out_v_mean_squared_error: 0.0

i: 72
Epoch 1/1
16/16 [==============================] - 0s 870us/step - loss: 5.6138 - out_p_loss: 5.3683 - out_v_loss: 0.0624 - out_p_categorical_crossentropy: 5.3683 - out_v_mean_squared_error: 0.0624
i: 73
Epoch 1/1
16/16 [==============================] - 0s 841us/step - loss: 5.6421 - out_p_loss: 5.3587 - out_v_loss: 0.1003 - out_p_categorical_crossentropy: 5.3587 - out_v_mean_squared_error: 0.1003
i: 74
Epoch 1/1
16/16 [==============================] - 0s 848us/step - loss: 5.6621 - out_p_loss: 5.4031 - out_v_loss: 0.0760 - out_p_categorical_crossentropy: 5.4031 - out_v_mean_squared_error: 0.0760
i: 75
Epoch 1/1
16/16 [==============================] - 0s 785us/step - loss: 5.5606 - out_p_loss: 5.3310 - out_v_loss: 0.0466 - out_p_categorical_crossentropy: 5.3310 - out_v_mean_squared_error: 0.0466
i: 76
Epoch 1/1
16/16 [==============================] - 0s 831us/step - loss: 5.5803 - out_p_loss: 5.3490 - out_v_loss: 0.0484 - out_p_categorical_crossentropy: 5.3490 - out_v_mean_sq

i: 7
Epoch 1/1
16/16 [==============================] - 0s 742us/step - loss: 5.6962 - out_p_loss: 5.4222 - out_v_loss: 0.0925 - out_p_categorical_crossentropy: 5.4222 - out_v_mean_squared_error: 0.0925
i: 8
Epoch 1/1
16/16 [==============================] - 0s 757us/step - loss: 5.6109 - out_p_loss: 5.3603 - out_v_loss: 0.0692 - out_p_categorical_crossentropy: 5.3603 - out_v_mean_squared_error: 0.0692
i: 9
Epoch 1/1
16/16 [==============================] - 0s 747us/step - loss: 5.6274 - out_p_loss: 5.4137 - out_v_loss: 0.0323 - out_p_categorical_crossentropy: 5.4137 - out_v_mean_squared_error: 0.0323
i: 10
Epoch 1/1
16/16 [==============================] - 0s 749us/step - loss: 5.6765 - out_p_loss: 5.4166 - out_v_loss: 0.0785 - out_p_categorical_crossentropy: 5.4166 - out_v_mean_squared_error: 0.0785
i: 11
Epoch 1/1
16/16 [==============================] - 0s 741us/step - loss: 5.7145 - out_p_loss: 5.3710 - out_v_loss: 0.1621 - out_p_categorical_crossentropy: 5.3710 - out_v_mean_squar

i: 48
Epoch 1/1
16/16 [==============================] - 0s 776us/step - loss: 5.6270 - out_p_loss: 5.3959 - out_v_loss: 0.0509 - out_p_categorical_crossentropy: 5.3959 - out_v_mean_squared_error: 0.0509
i: 49
Epoch 1/1
16/16 [==============================] - 0s 821us/step - loss: 5.5260 - out_p_loss: 5.3415 - out_v_loss: 0.0044 - out_p_categorical_crossentropy: 5.3415 - out_v_mean_squared_error: 0.0044
i: 50
Epoch 1/1
16/16 [==============================] - 0s 1ms/step - loss: 5.5762 - out_p_loss: 5.3451 - out_v_loss: 0.0509 - out_p_categorical_crossentropy: 5.3451 - out_v_mean_squared_error: 0.0509
i: 51
Epoch 1/1
16/16 [==============================] - 0s 875us/step - loss: 5.6348 - out_p_loss: 5.3917 - out_v_loss: 0.0630 - out_p_categorical_crossentropy: 5.3917 - out_v_mean_squared_error: 0.0630
i: 52
Epoch 1/1
16/16 [==============================] - 0s 862us/step - loss: 5.5788 - out_p_loss: 5.3799 - out_v_loss: 0.0189 - out_p_categorical_crossentropy: 5.3799 - out_v_mean_squa

i: 89
Epoch 1/1
16/16 [==============================] - 0s 878us/step - loss: 5.6214 - out_p_loss: 5.3824 - out_v_loss: 0.0604 - out_p_categorical_crossentropy: 5.3824 - out_v_mean_squared_error: 0.0604
i: 90
Epoch 1/1
16/16 [==============================] - 0s 753us/step - loss: 5.5803 - out_p_loss: 5.3436 - out_v_loss: 0.0581 - out_p_categorical_crossentropy: 5.3436 - out_v_mean_squared_error: 0.0581
i: 91
Epoch 1/1
16/16 [==============================] - 0s 795us/step - loss: 5.5245 - out_p_loss: 5.3183 - out_v_loss: 0.0276 - out_p_categorical_crossentropy: 5.3183 - out_v_mean_squared_error: 0.0276
i: 92
Epoch 1/1
16/16 [==============================] - 0s 784us/step - loss: 5.5746 - out_p_loss: 5.3461 - out_v_loss: 0.0499 - out_p_categorical_crossentropy: 5.3461 - out_v_mean_squared_error: 0.0499
i: 93
Epoch 1/1
16/16 [==============================] - 0s 869us/step - loss: 5.5534 - out_p_loss: 5.3433 - out_v_loss: 0.0316 - out_p_categorical_crossentropy: 5.3433 - out_v_mean_sq